In [0]:
%sql
-- Architecture Overview
/*
Data Lake + Data Warehouse = Modern Data Warehouse Architecture
Mô hình trên tiềm ẩn thách thức
- Duy trì và vận hành hai hệ thống khác nhau
- VD: 100% dữ liệu ở Data Lake, muốn đổ 20% data vào Warehouse -> Lặp dữ liệu
- Chi phí cao

Lakehouse ra đời để giải quyết bài toán trên, sử dụng một hệ thống có ưu điểm của cả Lake và Warehouse, giá rẻ nên dễ tiếp cận
- Lưu trữ được mọi loại dữ liệu
- Hỗ trợ mọi workload: BI, ML, AI

Kiến trúc của Databricks gồm ba tầng: Storage Layer -> Transactional Layer -> Delta Engine
Delta Engine là động cơ tăng tốc cho hai Layer bên dưới, nó là kiến trúc thượng tầng

Databricks Lakehouse không thay thế hoàn toàn DWH
*/

In [0]:
%sql
-- Delta Engine
/*

*/

In [0]:
%fs
ls dbfs:/databricks-datasets/nyctaxi/tripdata/yellow

In [0]:
df = spark.read \
    .format("csv") \
    .option("header","true") \
        .option("inferSchema", "true") \
            .load("dbfs:/databricks-datasets/nyctaxi/tripdata/yellow/yellow_tripdata_2009-01.csv.gz")


In [0]:
# Lấy config của Spark Apps built-in Databricks
for k, v in spark.sparkContext.getConf().getAll():
    print(f"{k} = {v}")


In [0]:
display(df)

In [0]:
%sql
-- Tạo Database
CREATE DATABASE IF NOT EXISTS taxi_db

In [0]:
%sql
DROP TABLE taxi_db.trip_delta;
    
DROP TABLE taxi_db.trip_parquet;

In [0]:
df.repartition(20) \
    .write \
        .mode("overwrite") \
        .format("delta") \
            .saveAsTable("taxi_db.trip_delta")

In [0]:
df.repartition(20) \
    .write \
        .format("parquet") \
            .saveAsTable("taxi_db.trip_parquet")

In [0]:
%fs
ls /user/hive/warehouse/taxi_db.db/trip_delta/_delta_log

In [0]:
df_read = spark.read.format("json").load("dbfs:/user/hive/warehouse/taxi_db.db/trip_delta/_delta_log/00000000000000000001.json")
display(df_read)
/*
Tái phân vùng thành 20 (repartition(20)) trước khi ghi dữ liệu nhằm giảm thiểu số lượng file khi ghi dữ liệu
-> Sinh ra 20 file parquet để Spark ghi
*/

In [0]:
%sql
EXPLAIN (SELECT min(Fare_Amt), max(Fare_Amt) FROM taxi_db.trip_parquet)

In [0]:
%sql
EXPLAIN (SELECT min(Fare_Amt), max(Fare_Amt) FROM taxi_db.trip_delta)

In [0]:
%sql
SELECT min(Fare_Amt), max(Fare_Amt) FROM taxi_db.trip_delta

In [0]:
%sql
SELECT min(Fare_Amt), max(Fare_Amt) FROM taxi_db.trip_parquet

In [0]:
%sql
-- Delta nhanh hơn nhiều so với Parquet
-- Tìm lý do, trước tiên EXPLAIN câu truy vấn để xem execution plan
/*
Với bảng Parquet, Spark phải đọc từ disk và full scan toàn bảng, tính toán (Aggerate) ra kết quả min,max
Với bảng Delta, Spark đọc thẳng vào metadata trong _delta_log, không tốn I/O xuống ổ đĩa
*/



In [0]:
%sql
SELECT * FROM taxi_db.trip_parquet  WHERE Total_Amt < 10;

In [0]:
%sql
SELECT * FROM taxi_db.trip_parquet  WHERE Total_Amt < 10;